In [ ]:
print("ashish")

In [1]:
import os

# Experiment configurations
rate_norm_values = [1, 0.9, 0.8]  # Change as needed
rate_dist_values = [0.1, 0.2, 0.3, 0.4]  # Change as needed
layer_begin_values = [0, 10]  # Change as needed
layer_end_values = [120, 140]  # Change as needed
layer_inter_values = [1, 2]  # Change as needed

# Output directory for experiment scripts
output_dir = "experiment_scripts"
os.makedirs(output_dir, exist_ok=True)

# Generate experiment scripts
for rate_norm in rate_norm_values:
    for rate_dist in rate_dist_values:
        for layer_begin in layer_begin_values:
            for layer_end in layer_end_values:
                for layer_inter in layer_inter_values:
                    # Skip invalid combinations
                    if layer_begin >= layer_end:
                        continue
                    if layer_inter > (layer_end - layer_begin):
                        continue
                    
                    # Create output directory name based on experiment configuration
                    output_subdir = f"RN{rate_norm}_RD{rate_dist}_LB{layer_begin}_LE{layer_end}_LI{layer_inter}"
                    output_path = os.path.join(output_dir, output_subdir)
                    os.makedirs(output_path, exist_ok=True)
                    
                    # Generate experiment script
                    script_content = f"""\
# Run the main script to train the model
!python -m torch.distributed.launch --nproc_per_node=3 --use_env main.py --coco_path ../open-images-bus-trucks/ \
  --epochs 20 --lr=1e-6 --batch_size=2 --num_workers=4 \
  --output_dir="{output_subdir}" --resume="detr-r50_no-class-head.pth" \
  --rate_norm {rate_norm} --rate_dist {rate_dist} --layer_begin {layer_begin} --layer_end {layer_end} --layer_inter {layer_inter}
"""
                    script_filename = os.path.join(output_path, "experiment_script.sh")
                    with open(script_filename, "w") as script_file:
                        script_file.write(script_content)

print("Experiment scripts generated successfully.")


Experiment scripts generated successfully.


In [6]:
import os

# Experiment configurations
rate_norm_values = [1, 0.9, 0.8]  # Change as needed
rate_dist_values = [0.1, 0.2, 0.3, 0.4]  # Change as needed
layer_begin_values = [0, 10]  # Change as needed
layer_end_values = [120, 140]  # Change as needed
layer_inter_values = [1, 2]  # Change as needed

# Output directory for experiment scripts
output_dir = "experiment_scripts"
os.makedirs(output_dir, exist_ok=True)

# Number of experiments per script
experiments_per_script = 8

# Counter for experiments
experiment_count = 0

# Generate experiment scripts
for rate_norm in rate_norm_values:
    for rate_dist in rate_dist_values:
        for layer_begin in layer_begin_values:
            for layer_end in layer_end_values:
                for layer_inter in layer_inter_values:
                    # Skip invalid combinations
                    if layer_begin >= layer_end:
                        continue
                    if layer_inter > (layer_end - layer_begin):
                        continue

                    # Check if a new script needs to be created
                    if experiment_count % experiments_per_script == 0:
                        script_index = experiment_count // experiments_per_script
                        script_filename = os.path.join(output_dir, f"experiment_script_{script_index}.sh")
                        script_content = ""

                    # Append experiment command to script content
                    output_subdir = f"RN{rate_norm}_RD{rate_dist}_LB{layer_begin}_LE{layer_end}_LI{layer_inter}"
                    experiment_command = f"""\
# Run experiment for {output_subdir}
!python -m torch.distributed.launch --nproc_per_node=3 --use_env main.py --coco_path ../open-images-bus-trucks/ \
  --epochs 200 --lr=1e-4 --batch_size=4 --num_workers=4 \
  --output_dir="{output_subdir}" --resume="detr-r50_no-class-head.pth" \
  --rate_norm {rate_norm} --rate_dist {rate_dist} --layer_begin {layer_begin} --layer_end {layer_end} --layer_inter {layer_inter}
"""
                    script_content += experiment_command
                    experiment_count += 1

                    # Write script content to file when it reaches experiments_per_script
                    if experiment_count % experiments_per_script == 0:
                        with open(script_filename, "w") as script_file:
                            script_file.write(script_content)

# Write the last script if there are remaining experiments
if experiment_count % experiments_per_script != 0:
    script_index = experiment_count // experiments_per_script
    script_filename = os.path.join(output_dir, f"experiment_script_{script_index}.sh")
    with open(script_filename, "w") as script_file:
        script_file.write(script_content)

print("Experiment scripts generated successfully.")


Experiment scripts generated successfully.
